## Homework

In this homework, we'll take the ride duration prediction model
that we deployed in batch mode in homework 4 and improve the 
reliability of our code with unit and integration tests. 

You'll find the starter code in the [homework](homework/) directory.


## Q1. Refactoring

Before we can start converting our code with tests, we need to 
refactor it. We'll start by getting rid of all the global variables. 

* Let's create a function `main` with two parameters: `year` and
`month`.
* Move all the code (except `read_data`) inside `main`
* Make `categorical` a parameter for `read_data` and pass it inside `main`

Now we need to create the "main" block from which we'll invoke
the main function. How does the `if` statement that we use for
this looks like? 


Hint: after refactoring, check that the code still works. Just run it e.g. for March 2023 and see if it finishes successfully. 

To make it easier to run it, you can write results to your local
filesystem. E.g. here:

```python
output_file = f'taxi_type=yellow_year={year:04d}_month={month:02d}.parquet'
```


In [ ]:
# 1. '!cd 2024_mlops_homework/06-best-practices/2024/homework'
# 2. Refactor code and name it 'refactor_batch.py':
import sys
import pickle
import pandas as pd

def read_data(filename, categorical):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

def main(year, month):
    input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
    output_file = f'taxi_type=yellow_year={year:04d}_month={month:02d}.parquet'

    with open('model.bin', 'rb') as f_in:
        dv, lr = pickle.load(f_in)

    categorical = ['PULocationID', 'DOLocationID']
    df = read_data(input_file, categorical)
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = lr.predict(X_val)

    print('predicted mean duration:', y_pred.mean())

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred

    df_result.to_parquet(output_file, engine='pyarrow', index=False)

if __name__ == "__main__":
    year = int(sys.argv[1])
    month = int(sys.argv[2])
    main(year, month)

# 3. 'python refactor_batch_q1.py 2023 3'
# --> 'if __name__ == "__main__":'

## Q2. Installing pytest

Now we need to install `pytest`:

```bash
pipenv install --dev pytest
```

Next, create a folder `tests` and create two files. One will be
the file with tests. We can name it `test_batch.py`. 

What should be the other file? 

Hint: to be able to test `batch.py`, we need to be able to
import it. Without this other file, we won't be able to do it.

In [ ]:
# 1. 'cd /2024_mlops_homework/06-best-practices/2024/homework'
# 2. 'pipenv install --dev pytest'
# 3. 'pipenv --venv' to retrieve <venv_dir>
# 4. Press Shift+P, type 'Python: Interpreter Select', and insert <venv_dir>/bin/python (e.g., /home/pxmopsadmin/.local/share/virtualenvs/homework-ZTKjIiBs/bin/python)
# 5. Press Shift+P, type 'Python: Interpreter Select', and choose <venv> (e.g., homework-ZTKjIiBs)
# 6. 'pipenv shell' (inside '/2024_mlops_homework/06-best-practices/2024/homework'!)
# 7. Create 'tests' folder 
# 8. Insert new file 'test_batch.py'
# 9. Insert __init__.py
# --> '__init__.py'

## Q3. Writing first unit test

Now let's cover our code with unit tests.

We'll start with the pre-processing logic inside `read_data`.

It's difficult to test right now because first reads
the file and then performs some transformations. We need to split this 
code into two parts: reading (I/O) and transformation. 

So let's create a function `prepare_data` that takes in a dataframe 
(and some other parameters too) and applies some transformation to it.

(That's basically the entire `read_data` function after reading 
the parquet file)

Now create a test and use this as input:

```python
data = [
    (None, None, dt(1, 1), dt(1, 10)),
    (1, 1, dt(1, 2), dt(1, 10)),
    (1, None, dt(1, 2, 0), dt(1, 2, 59)),
    (3, 4, dt(1, 2, 0), dt(2, 2, 1)),      
]

columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime']
df = pd.DataFrame(data, columns=columns)
```

Where `dt` is a helper function:

```python
from datetime import datetime

def dt(hour, minute, second=0):
    return datetime(2023, 1, 1, hour, minute, second)
```

Define the expected output and use the assert to make sure 
that the actual dataframe matches the expected one.

Tip: When you compare two Pandas DataFrames, the result is also a DataFrame.
The same is true for Pandas Series. Also, a DataFrame could be turned into a list of dictionaries.  

How many rows should be there in the expected dataframe?

* 1
* 2
* 3
* 4

In [ ]:
# 1. Adjust 'refactor_batch.py' to split 'def read_data(filename, categorical)' into two parts: 'def read_data(filename)' and 'def prepare_data(df, categorical)'
# 2. Create unit test using the 'test_batch.py' file created in Q2.
#   a. 'import refactor_batch' (must lie one dir above test script!)
#   b. Input test data and call 'prepare_data'
#   c. Input expected data and 'assert'
# 3. 'cd /2024_mlops_homework/06-best-practices/2024/homework/tests'
# 4. 'pytest test_prepare_data.py'
# --> 4 rows are inserted, 2 ROWS ARE EXPECTED. Reason: Row 3 and 4 are dropped as duration <1 and >60.

## Q4. Mocking S3 with Localstack 

Now let's prepare for an integration test. In our script, we 
write data to S3. So we'll use Localstack to mimic S3.

First, let's run Localstack with Docker compose. Let's create a 
`docker-compose.yaml` file with just one service: localstack. Inside
localstack, we're only interested in running S3. 

Start the service and test it by creating a bucket where we'll
keep the output. Let's call it "nyc-duration".

With AWS CLI, this is how we create a bucket:

```bash
aws s3 mb s3://nyc-duration
```

Then we need to check that the bucket was successfully created. With AWS, this is how we typically do it:

```bash
aws s3 ls
```

In both cases we should adjust commands for localstack. What option do we need to use for such purposes?

* `--backend-store-uri`
* `--profile`
* `--endpoint-url`
* `--version`


## Make input and output paths configurable

Right now the input and output paths are hardcoded, but we want
to change it for the tests. 

One of the possible ways would be to specify `INPUT_FILE_PATTERN` and `OUTPUT_FILE_PATTERN` via the env 
variables. Let's do that:


```bash
export INPUT_FILE_PATTERN="s3://nyc-duration/in/{year:04d}-{month:02d}.parquet"
export OUTPUT_FILE_PATTERN="s3://nyc-duration/out/{year:04d}-{month:02d}.parquet"
```

And this is how we can read them:

```python
def get_input_path(year, month):
    default_input_pattern = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
    input_pattern = os.getenv('INPUT_FILE_PATTERN', default_input_pattern)
    return input_pattern.format(year=year, month=month)


def get_output_path(year, month):
    default_output_pattern = 's3://nyc-duration-prediction-alexey/taxi_type=fhv/year={year:04d}/month={month:02d}/predictions.parquet'
    output_pattern = os.getenv('OUTPUT_FILE_PATTERN', default_output_pattern)
    return output_pattern.format(year=year, month=month)


def main(year, month):
    input_file = get_input_path(year, month)
    output_file = get_output_path(year, month)
    # rest of the main function ... 
```


## Reading from Localstack S3 with Pandas

So far we've been reading parquet files from S3 with using
pandas `read_parquet`. But this way we read it from the
actual S3 service. Now we need to replace it with our localstack
one.

For that, we need to specify the endpoint url:

```python
options = {
    'client_kwargs': {
        'endpoint_url': S3_ENDPOINT_URL
    }
}

df = pd.read_parquet('s3://bucket/file.parquet', storage_options=options)
```

Let's modify our `read_data` function:

- check if `S3_ENDPOINT_URL` is set, and if it is, use it for reading
- otherwise use the usual way

In [6]:
# 0. Copy 'refactor_batch_q4.py' from 'refactor_batch_q3.py'
# 1. Create dockerfile
        # FROM python:3.10.9-slim

        # # Install CA certificates and update certificates bundle
        # RUN apt-get update && apt-get install -y --no-install-recommends ca-certificates && update-ca-certificates

        # RUN pip install -U pip & pip install pipenv

        # COPY [ "Pipfile", "Pipfile.lock", "./" ]

        # RUN pipenv install --system --deploy

        # COPY [ "refactor_batch_q4.py", "refactor_batch_q4.py" ]
        # COPY [ "model.bin", "model.bin" ]

        # ENTRYPOINT [ "python", "refactor_batch_q3.py" ]
# 2. Refactor 'refactor_batch_q4.py'
        # def read_data(filename):
        #     s3_endpoint_url = os.getenv('S3_ENDPOINT_URL')
        #     if s3_endpoint_url:
        #         options = {'client_kwargs': {'endpoint_url': s3_endpoint_url}}
        #         df = pd.read_parquet(filename, storage_options=options)
        #     else:
        #         input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
        #         df = pd.read_parquet(input_file)
        #     return df

        # ... # rest of prepare data

        # def get_input_path(year, month):
        #     default_input_pattern = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
        #     input_pattern = os.getenv('INPUT_FILE_PATTERN', default_input_pattern)
        #     return input_pattern.format(year=year, month=month)

        # def get_output_path(year, month):
        #     default_output_pattern = 's3://nyc-duration-prediction-alexey/taxi_type=fhv/year={year:04d}/month={month:02d}/predictions.parquet'
        #     output_pattern = os.getenv('OUTPUT_FILE_PATTERN', default_output_pattern)
        #     return output_pattern.format(year=year, month=month)

        # def main(year, month):
        #     input_file = get_input_path(year, month)
        #     output_file = get_output_path(year, month)

        # ... # rest of main

        #     s3_endpoint_url = os.getenv('S3_ENDPOINT_URL')
        #     if s3_endpoint_url:
        #         options = {'client_kwargs': {'endpoint_url': s3_endpoint_url}} 
        #         df_result.to_parquet(output_file, engine='pyarrow', index=False, storage_options=options)
        #     else: print('s3_endpoint_url not set. Output not written.')

        # ... # rest of main
                
# 3. Create dir 'integration-test' dir should look like this now:
        # project-root/
        # ├── Dockerfile
        # ├── Pipfile
        # ├── Pipfile.lock
        # ├── refactor_batch_q3.py
        # └── model.bin
        # ├──── integration-test/
        # ├──── unit-test/
        #      ├── __init__.py
        #      ├── test_batch.py
# 3. Create 'docker-compose.yaml' inside 'integration-test'
        # services:
        #   localstack:
        #     image: localstack/localstack
        #     ports:
        #       - "127.0.0.1:4566:4566"
        #     environment:
        #       - SERVICES=s3
        #       - INPUT_FILE_PATTERN=${INPUT_FILE_PATTERN}
        #       - OUTPUT_FILE_PATTERN=${OUTPUT_FILE_PATTERN}
        #       - DEFAULT_REGION=eu-west-1
        #       - AWS_ACCESS_KEY_ID=abc
        #       - AWS_SECRET_ACCESS_KEY=xyz
        #       - S3_ENDPOINT_URL=http://localhost:4566
        #     volumes:
        #       - "../model:/app/model"
# 4.  Create 'run.sh' inside 'integration-test'
        # #!/usr/bin/env bash

        # # Define the image name and tag
        # export LOCAL_IMAGE_NAME="s3-model-duration:latest"

        # # Build the Docker image
        # docker build -t ${LOCAL_IMAGE_NAME} ../

        # # Export environment variables for Docker Compose
        # export PREDICTIONS_S3_NAME="ride_predictions"
        # export INPUT_FILE_PATTERN="s3://nyc-duration/in/{year:04d}-{month:02d}.parquet"
        # export OUTPUT_FILE_PATTERN="s3://nyc-duration/out/{year:04d}-{month:02d}.parquet"
        # export S3_ENDPOINT_URL="http://localhost:4566"
        # export DEFAULT_REGION="eu-west-1"
        # export AWS_ACCESS_KEY_ID="abc"
        # export AWS_SECRET_ACCESS_KEY="xyz"

        # docker-compose up -d
# 5. Bash chmod +x run.sh
# 6. Start docker with ./run.sh
# 7. If not done already: 'sudo apt install awscli'
# 8. Add AWS credentials using 'sudo nano ~/.aws/credentials'
        # [default]
        # aws_access_key_id=abc
        # aws_secret_access_key=xyz
# 9. Add AWS config using 'sudo nano ~/.aws/config'
        # [default]
        # region = eu-west-1
# 10. Create S3 Bucket with AWS CLI 'sudo aws --endpoint-url=http://localhost:4566 s3 mb s3://nyc-duration'
# 10. Verify bucket was created: 'sudo aws --endpoint-url=http://localhost:4566 s3 ls'
# 11. If successful, add 'sleep 5' and 'sudo aws --endpoint-url=http://localhost:4566 s3 mb s3://nyc-duration' to 'run.sh' 

## Q5. Creating test data

Now let's create `integration_test.py`

We'll use the dataframe we created in Q3 (the dataframe for the unit test)
and save it to S3. You don't need to do anything else: just create a dataframe 
and save it.

We will pretend that this is data for January 2023.

Run the `integration_test.py` script. After that, use AWS CLI to verify that the 
file was created. 

Use this snipped for saving the file:

```python
df_input.to_parquet(
    input_file,
    engine='pyarrow',
    compression=None,
    index=False,
    storage_options=options
)
```

What's the size of the file?

* 3620
* 23620
* 43620
* 63620

Note: it's important to use the code from the snippet for saving
the file. Otherwise the size may be different depending on the OS,
engine and compression. Even if you use this exact snippet, the size
of your dataframe may still be a bit off. Just select the closest option.

In [ ]:
# 1. create 'integration_test.py'
    # import pandas as pd
    # from datetime import datetime
    # import os
    # import sys

    # def dt(hour, minute, second=0):
    #     return datetime(2023, 1, 1, hour, minute, second)

    # def get_input_path(year, month):
    #     default_input_pattern = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
    #     input_pattern = os.getenv('INPUT_FILE_PATTERN', default_input_pattern)
    #     return input_pattern.format(year=year, month=month)

    # def main(year, month):
    #     input_file = get_input_path(year, month)
    #     data = [
    #             (None, None, dt(1, 1), dt(1, 10)),
    #             (1, 1, dt(1, 2), dt(1, 10)),
    #             (1, None, dt(1, 2, 0), dt(1, 2, 59)),
    #             (3, 4, dt(1, 2, 0), dt(2, 2, 1)),      
    #         ]
    #     columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime']
    #     df = pd.DataFrame(data, columns=columns)
    #     s3_endpoint_url = os.getenv('S3_ENDPOINT_URL')
    #     options = {'client_kwargs': {'endpoint_url': s3_endpoint_url}}
    #     print(f'input_file: {input_file} | s3_endpoint_url: {s3_endpoint_url} ')
    #     df.to_parquet(
    #         input_file,
    #         engine='pyarrow',
    #         compression=None,
    #         index=False,
    #         storage_options=options
    #     )

    # if __name__ == "__main__":
    #     year = int(sys.argv[1])
    #     month = int(sys.argv[2])
    #     main(year, month)
# 2. Add 'pipenv run python integration_test.py 2023 1' to 'run.sh'
# 3. Get files and sizes 'aws --endpoint-url=http://localhost:4566 s3 ls s3://nyc-duration --recursive'
# --> 2024-06-27 13:57:55       3620 in/2023-01.parquet

## Q6. Finish the integration test

We can read from our localstack s3, but we also need to write to it.

Create a function `save_data` which works similarly to `read_data`,
but we use it for saving a dataframe. 

Let's run the `batch.py` script for January 2023 (the fake data
we created in Q5). 

We can do that from our integration test in Python: we can use
`os.system` for doing that (there are other options too). 

Now it saves the result to localstack.

The only thing we need to do now is to read this data and 
verify the result is correct. 

What's the sum of predicted durations for the test dataframe?

* 13.08
* 36.28
* 69.28
* 81.08